# Homework 5 RF Accuracy Improvement

This assignment is inspired by examples of Shan-Hung Wu from National Tsing Hua University.

Requirement: improve the accuracy per feature of the following code from 0.03 up to at least 0.45 and accuracy should be more than 0.92

Here are three hints:

    You can improve the ratio by picking out or "creating" several features.
    Tune hyperparameters
    The ratio can be improved from 0.03 up to 0.47.

In [ ]:
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.datasets import load_breast_cancer
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.model_selection import RandomizedSearchCV
from sklearn.feature_selection import RFE
from sklearn.metrics import accuracy_score
from scipy.stats import uniform
import numpy as np

# load the breast_cancer dataset
init_data = load_breast_cancer()
(X, y) = load_breast_cancer(return_X_y=True)

# TODO Select some features (X), hint: based on the connections with our Y (importance? correlation?)

# TODO need 5 fold cross validation

from sklearn.inspection import permutation_importance

rf_classifier = RandomForestClassifier()
rf_classifier.fit(X,y)
result = permutation_importance(rf_classifier, X, y, n_repeats=30, random_state=42)
importances = result.importances_mean
top_k_indices = np.argsort(importances)[-k:]
X = X[:, top_k_indices]

param_dist = {
    'n_estimators': [10,50,100,200,300],
    'criterion': ['gini', 'entropy'],
    'max_depth': [None, 10, 20, 30, 40, 50],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'max_features': ['auto', 'sqrt', 'log2', None],
    'bootstrap': [True, False],
    'class_weight': [None, 'balanced', 'balanced_subsample']
}

randm_src = RandomizedSearchCV(
    rf_classifier,      # The Lasso model
    param_distributions=param_dist,
    cv=5
)
randm_src.fit(X, y)

print(randm_src.best_params_)
print(randm_src.best_score_)


cv_scores = cross_val_score(RandomForestClassifier(**randm_src.best_params_), X, y, cv=5)


# TODO Calculate Average accuracy score
average_accuracy = np.mean(cv_scores)

# TODO Calculate Average (accuracy score/number of features)
average_accuracy_per_feature = average_accuracy / X.shape[1]

# Print the results
print("Average Accuracy: {:.2f}%".format(average_accuracy * 100))
print("Average Accuracy per Feature: {:.2f}%".format(average_accuracy_per_feature * 100))
